In [2]:
# =========================================================================
# Copyright (C) 2023. Huawei Technologies Co., Ltd. All rights reserved.
# 
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# =========================================================================


import os
# os.chdir(os.path.dirname(os.path.realpath(__file__)))
import sys
import logging
import fuxictr_version
from datetime import datetime
from fuxictr.utils import load_config, set_logger, print_to_json, print_to_list
from fuxictr.features import FeatureMap
from fuxictr.tensorflow.tf_utils import seed_everything
from fuxictr.tensorflow.dataloaders import TFRecordDataLoader
import tensorflow as tf
import src as model_zoo
import gc
import argparse
import os
from pathlib import Path

# if __name__ == '__main__':
''' Usage: python run_expid.py --config {config_dir} --expid {experiment_id} --gpu {gpu_device_id}
'''
parser = argparse.ArgumentParser()
parser.add_argument('--config', type=str, default='./config/', help='The config directory.')
parser.add_argument('--expid', type=str, default='DCN_test', help='The experiment id to run.')
parser.add_argument('--gpu', type=int, default=0, help='The gpu index, -1 for cpu')
args = vars(parser.parse_args())

gpus = tf.config.experimental.list_physical_devices('GPU')
if args['gpu'] >= 0:
    tf.config.set_visible_devices(gpus[args['gpu']], 'GPU')
    tf.config.experimental.set_memory_growth(gpus[args['gpu']], True)
experiment_id = args['expid']
params = load_config(args['config'], experiment_id)
set_logger(params)
logging.info("Params: " + print_to_json(params))
seed_everything(seed=params['seed'])

data_dir = os.path.join(params['data_root'], params['dataset_id'])
feature_map_json = os.path.join(data_dir, "feature_map.json")
feature_map = FeatureMap(params['dataset_id'], data_dir)
feature_map.load(feature_map_json, params)
logging.info("Feature specs: " + print_to_json(feature_map.features))

model_class = getattr(model_zoo, params['model'])
model = model_class(feature_map, **params)

train_gen, valid_gen = TFRecordDataLoader(feature_map, stage='train', **params).make_iterator()
model.fit(train_gen, validation_data=valid_gen, **params)

logging.info('****** Validation evaluation ******')
valid_result = model.evaluate(valid_gen)
del train_gen, valid_gen
gc.collect()

logging.info('******** Test evaluation ********')
test_gen = TFRecordDataLoader(feature_map, stage='test', **params).make_iterator()
test_result = {}
if test_gen:
    test_result = model.evaluate(test_gen)
result_filename = Path(args['config']).name.replace(".yaml", "") + '.csv'
model.save(filepath="./model")
with open(result_filename, 'a+') as fw:
    fw.write(' {},[command] python {},[exp_id] {},[dataset_id] {},[train] {},[val] {},[test] {}\n' \
        .format(datetime.now().strftime('%Y%m%d-%H%M%S'), 
                ' '.join(sys.argv), experiment_id, params['dataset_id'],
                "N.A.", print_to_list(valid_result), print_to_list(test_result)))



fuxictr.__version__=2.0.0


2023-03-26 19:44:47.238493: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-26 19:44:47.318258: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-26 19:44:47.623924: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-26 19:44:47.623953: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not l

SystemExit: 2

/home/zhuangzhong/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
